# Estimation Tutorial

In this section, we dive into the topic of model estimation using **pydsge**. 

Let us, just for the sake of this tutorial, set up a temporary directory structure:

In [1]:
# Just for the tutorial: Setting up example structure
import tempfile
import os
import shutil # For clean-up of temporary directory
from pathlib import Path # For Windows/Unix compatibility

# Temporary output folder
output_path = Path(tempfile.gettempdir(), 'output')
if not os.path.isdir(output_path):
    os.makedirs(output_path)

## Parsing and loading the model

Let us first load the relevant packages. Besides the DSGE class we already know from [*getting started*](https://pydsge.readthedocs.io/en/latest/getting_started.html), we also want to import the `emcee` package. This will allow us to later specify the desired updating algorithms for sampling from the posterior distribution - we explain this in more detail below.

In [2]:
import pandas as pd
import numpy as np
import emcee # For specifying updating moves

from pydsge import DSGE, example

In this tutorial, we continue to use the example provided in `pydsge`. Like before, we specify the file paths of the model and the data. Please feel free to check-out both files, but from the previous tutorial you might remember that we're dealing with a five equations New Keynesian model and US quarterly data from 1995 to 2018. 

In [3]:
yaml_file, data_file = example

We again parse the model and load the data. Importantly we also specify a location where the output is stored. 

For this tutoral I assign a tempfile as the output path, but this is certainly not what you want. So better change that path. Note also that we can give the model estimation a name and also write a short description, which is very useful when working with several models or estimations.

In [4]:
# Parse the model
mod = DSGE.read(yaml_file)  

# Give it a name
mod.name = 'rank_tutorial'
mod.description = 'RANK, estimation tutorial'

# Storage location for output. CHANGE THIS to some local path on your PC!
mod.path = output_path

# Load data
df = pd.read_csv(data_file, parse_dates=['date'], index_col=['date'])
df.index.freq = 'Q' # let pandas know that this is quartely data

Remember that since the Great Recession, the Federal Funds Rate has been below the ZLB. That is why, like in [*getting started*](https://pydsge.readthedocs.io/en/latest/getting_started.html), we adjust the observed interest rate, so that the data is "within reach" of our model.

In [5]:
# adjust elb
zlb = mod.get_par('elb_level')
rate = df['FFR']
df['FFR'] = np.maximum(rate,zlb)

mod.load_data(df, start='1998Q1')

,GDP,Infl,FFR
date,,,
1998-03-31,0.77834,0.14386,1.38
1998-06-30,0.69635,0.22873,1.38
1998-09-30,1.03077,0.36109,1.38
1998-12-31,1.37921,0.26145,1.22
1999-03-31,0.54307,0.37393,1.18
...,...,...,...
2017-03-31,0.41475,0.49969,0.18
2017-06-30,0.54594,0.25245,0.24
2017-09-30,0.54391,0.51972,0.29


## Preparing the estimation

After importing the packages and loading the data, we still need to tell pydsge how to carry out the estimation of our model. The "prep_estim" method can be used to accomplish this. It can be called without any arguments and sets-up a non-linear model by default. However, not all defaults are always a good good choice, and to showcase some of this functionality, we decide to specify several arguments here.

To perform the estimation, `pydsge` uses a Transposed-Ensemble Kalman Filter (TEnKF). For general information on its implementation, see the [EconSieve documentation](https://econsieve.readthedocs.io/en/latest/) , and for more details on running the filter in `pydsge` check-out the [*getting started tutorial*](https://pydsge.readthedocs.io/en/latest/getting_started.html). Again,  the default filter is non-linear, but we can opt for a linear one by setting the argument `linear` to `True`. To choose a custom number of ensemble members for the TEnKF, set `N` to a particular number (default is 300, for e.g. a medium scale model 400-500 is a good choice). We can also set a specific random seed with the argument `seed` (the default seed is `0`). To get additional information on the estimation process, we can set  `verbose` to `True`. Conveniently, this information includes an overview of the parameters’ distribution, their means and standard deviations. Finally, if we already specified the covariance matrix of the measurement errors or want to reuse a previous result, we can load it into the `prep_estim` method by setting `Load.R` to `True`. 

If you run into problems you can turn parallelization off by setting `debug=True`.

In [6]:
mod.prep_estim(N=350, seed=0, verbose=True, use_prior_transform=True)

[estimation:]   Model operational. 12 states, 3 observables, 3 shocks, 81 data points.
Adding parameters to the prior distribution...
   - theta as beta (0.5, 0.1). Init @ 0.7813, with bounds (0.2, 0.95)
   - sigma as normal (1.5, 0.375). Init @ 1.2312, with bounds (0.25, 3)
   - phi_pi as normal (1.5, 0.25). Init @ 1.7985, with bounds (1.0, 3)
   - phi_y as normal (0.125, 0.05). Init @ 0.0893, with bounds (0.001, 0.5)
   - rho_u as beta (0.5, 0.2). Init @ 0.7, with bounds (0.01, 0.9999)
   - rho_r as beta (0.5, 0.2). Init @ 0.7, with bounds (0.01, 0.9999)
   - rho_z as beta (0.5, 0.2). Init @ 0.7, with bounds (0.01, 0.9999)
   - rho as beta (0.75, 0.1). Init @ 0.8, with bounds (0.5, 0.975)
   - sig_u as inv_gamma_dynare (0.1, 2). Init @ 0.5, with bounds (0.025, 5)
   - sig_r as inv_gamma_dynare (0.1, 2). Init @ 0.5, with bounds (0.01, 3)
   - sig_z as inv_gamma_dynare (0.1, 2). Init @ 0.5, with bounds (0.01, 3)
[estimation:]   11 priors detected. Adding parameters to the prior distrib

You probably want to set `use_prior_transform` to `True`, as above. This activates the destinction between prior and sampling space for the [ADEMC sampler](https://gregorboehl.com/live/ademc_boehl.pdf), which reduces the number of neccessary MCMC draws *a lot*.

As in the filtering tutorial, we set the covariance of measurement errors to correspond to the variances of the data. Additionally, we adjust the measurement errors of the Federal Funds rate since it is perfectly observable.

In [7]:
mod.filter.R = mod.create_obs_cov(1e-1)
ind = mod.observables.index('FFR')
mod.filter.R[ind,ind] /= 1e1 

## Running the estimation

Lets turn to the actual estimation. For a variety of pretty good reasons, `pdygse` uses *Ensemble Markov Chain Monte Carlo* (Ensemble-MCMC) integration to sample from the posterior distribution. For further information on Ensemble-MCMC, please refer to the `emcee` [website](https://emcee.readthedocs.io/en/stable/) and the additional resources provided there. 

We first require an initial ensemble, which is provided by `tmcmc`. `tmcmc` is a very sophisticated function with many options, but right now, all we are interested in is to obtain a sample that represents the prior distribution:

In [8]:
p0 = mod.prior_sampler(40, verbose=True) # rule of thumb: number_of_parameters times 4 

100%|██████████| 40/40 [00:03<00:00, 12.16it/s]

(prior_sample:) Sampling done. Check fails for 2.44% of the prior.


The parameter draws are saved in the object `p0` as a numpy array in order to later pass them to our main sampling process.

In [9]:
mod.save()

[save_meta:]    Metadata saved as '/tmp/output/Rank_tutorial_meta'


`mod.save()` saved the meta data of our model in the directory which we specified earlier in `mod.path`. This information is stored as an `.npz` file so that it is avialable even in the event of a crash and can be loaded anytime using `numpy.load()`.

For posterior sampling using `mcmc` we have the option to set different "moves", i.e. coordinate updating algorithms for the walkers. As a wrapper for a lot of `emcee` functionality,  `mcmc` can work with many different "moves" - for a list and implementation details please consult the `emcee` documentation. For using them here, specify them as a list of tuples, containing the type of move and its "weight". If no move is specified, `StretchMove` is used. 

In [10]:
moves = [(emcee.moves.DEMove(), 0.8), 
         (emcee.moves.DESnookerMove(), 0.2),]

We now use the initial states derived above to conduct our full Bayesian estimation using `mcmc`. Note that, instead of using the specified initial ensemble, `mcmc` can identify previous runs or estimations, or the initial values of the "prior" section in the `*.yaml` can be used. 

The default number of sampling steps is 3000, which is parallelized by default. With `tune` we can determine the size of the Markov Chain we wish to retain to represent the posterior, i.e. after burn-in. This is not to be confused this with the updating frequency, which only affects the number of summary statements `pydsge`reports during the estimation. 

With the option `lprob_seed` the user can choose how to set the random seed of the likelihood evaluation - here we use the seed specified in `prep_estim`. 

In [11]:
mod.mcmc(p0,
         moves=moves,
         nsteps=400,
         tune=100,
         update_freq=50,
         ) # this may take some time. Better run on a machine with MANY cores...
mod.save() # be sure to save the internal state!

[mcmc:]         HDF backend at /tmp/output/Rank_tutorial_sampler.h5 already exists. Deleting...


[ll/MAF:-80.0160360(4e+01)/3%]:  13%|█▎        | 51/400 [01:33<10:15,  1.76s/sample(s)]  

(mcmc:) Summary from last 50 of 50 iterations (RANK, estimation tutorial):
            distribution  pst_mean  sd/df    mean       sd   mode  hpd_5  \
theta               beta     0.500  0.100   0.749    0.146  0.601  0.584   
sigma             normal     1.500  0.375   1.861    0.971  2.153  0.696   
phi_pi            normal     1.500  0.250   1.715    0.449  1.370  0.920   
phi_y             normal     0.125  0.050   0.255    0.227  0.087 -0.069   
rho_u               beta     0.500  0.200   0.737    0.096  0.606  0.593   
rho_r               beta     0.500  0.200   0.511    0.156  0.588  0.233   
rho_z               beta     0.500  0.200   0.465    0.259  0.680  0.006   
rho                 beta     0.750  0.100   0.661    0.079  0.687  0.521   
sig_u   inv_gamma_dynare     0.100  2.000  15.360  219.483  1.034  0.520   
sig_r   inv_gamma_dynare     0.100  2.000   0.872    0.239  1.037  0.468   
sig_z   inv_gamma_dynare     0.100  2.000   0.547    0.382  1.055  0.095   

        hpd_

[ll/MAF:-30.7655438(4e+01)/3%]:  24%|██▍       | 95/400 [02:52<08:58,  1.77s/sample(s)] 

[preprocess:]   WARNING: at least one control indetermined


...

(mcmc:) Summary from last 50 of 300 iterations (RANK, estimation tutorial):
            distribution  pst_mean  sd/df   mean     sd   mode  hpd_5  hpd_95  \
theta               beta     0.500  0.100  0.921  0.087  0.965  0.789   0.994   
sigma             normal     1.500  0.375  2.481  0.433  2.061  1.810   3.181   
phi_pi            normal     1.500  0.250  1.374  0.770  0.745  0.455   2.534   
phi_y             normal     0.125  0.050  0.181  0.088  0.194  0.067   0.268   
rho_u               beta     0.500  0.200  0.947  0.020  0.942  0.929   0.973   
rho_r               beta     0.500  0.200  0.672  0.231  0.777  0.142   0.932   
rho_z               beta     0.500  0.200  0.632  0.265  0.579  0.351   0.998   
rho                 beta     0.750  0.100  0.775  0.080  0.759  0.674   0.892   
sig_u   inv_gamma_dynare     0.100  2.000  0.215  0.052  0.202  0.147   0.278   
sig_r   inv_gamma_dynare     0.100  2.000  0.097  0.015  0.088  0.080   0.125   
sig_z   inv_gamma_dynare     0.10

[ll/MAF:11.6914374(7e+00)/5%]:  88%|████████▊ | 351/400 [11:21<01:23,  1.71s/sample(s)] 

(mcmc:) Summary from last 50 of 350 iterations (RANK, estimation tutorial):
            distribution  pst_mean  sd/df   mean     sd   mode  hpd_5  hpd_95  \
theta               beta     0.500  0.100  0.919  0.087  0.986  0.783   0.986   
sigma             normal     1.500  0.375  2.380  0.428  2.943  1.741   3.048   
phi_pi            normal     1.500  0.250  1.290  0.782  0.950  0.390   2.545   
phi_y             normal     0.125  0.050  0.199  0.074  0.157  0.070   0.280   
rho_u               beta     0.500  0.200  0.945  0.014  0.956  0.927   0.968   
rho_r               beta     0.500  0.200  0.715  0.225  0.677  0.490   0.996   
rho_z               beta     0.500  0.200  0.623  0.266  0.503  0.345   0.999   
rho                 beta     0.750  0.100  0.753  0.081  0.763  0.667   0.892   
sig_u   inv_gamma_dynare     0.100  2.000  0.212  0.041  0.213  0.149   0.275   
sig_r   inv_gamma_dynare     0.100  2.000  0.092  0.013  0.104  0.071   0.109   
sig_z   inv_gamma_dynare     0.10

[ll/MAF:16.3164701(4e+00)/18%]: 100%|██████████| 400/400 [13:00<00:00,  1.95s/sample(s)]

[save_meta:]    Metadata saved as '/tmp/output/Rank_tutorial_meta'


Great. On my 8 core machine this just took 13 Minutes. 

So where are our estimates? Our (hopefully) converged MCMC samples are currently stored in the file named `rank_tutorial_sampler.h5` created by `mcmc` which is stored in the `output_path`. 

You can load and use this data using the methods introduced in the [*processing estimation results tutorial*](https://pydsge.readthedocs.io/en/latest/getting_started.html).

In [12]:
# Just for the tutorial: Cleaning the temporary directory
shutil.rmtree(output_path)